In [7]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
from string import punctuation
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F 
import torch.utils.data
import nltk
# nltk.download('stopwords')
nltk.download('punkt')

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')


from keyword_extraction import RakeKeywordExtractor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rake = RakeKeywordExtractor()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jz2av\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# test of sentence extraction
string = "Because we all get caught up in all the drama"
keywords = rake.sen_extract(string, incl_scores = True, word_num=1) # incl_scores: display the score, word_num: number of keywords
print(keywords[0][1])

def SigKeywordExtract(sentence):
    keywords = rake.sen_extract(sentence, incl_scores = True, word_num=1)
    if not keywords:
        return None;
    else:
        return keywords

def MutiKeywordExtract(sentence, num):
    keywords = rake.sen_extract(sentence, incl_scores = True, word_num=num)
    res = []
    for keyword in keywords:
        res.append(keyword[0])
    return res
print(string)
print(SigKeywordExtract(string))
print(MutiKeywordExtract(string, 2))

32.33333333333333
Because we all get caught up in all the drama
[('caught', 32.33333333333333)]
['caught', 'drama']


Firstly
We define a class to build corpus which contains wordEmbedding function (word2Index and Index2word).
In the class, we provide a function to calculate the number of tokens in the training data.

In [9]:
# Class lang: Convert each word to index and convert each index to word
# WordEmbedding - this structure can be viewed as a corpus 
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 #count SOS and EOS
        
    def addSentence(self, sentence):
        if sentence:
             for word in sentence.split(' '):
                self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Secondly
We process the training text data, load the data, convert the csv file into list[pair]. 
For every pair in the list, pair[0] is the current sentence(Encoder input), pair[1] is the next sentence(ground truth)

In [14]:
#Prepare for training data
#Convert all Unicode to ASCII, normalize the word: make everything lowercase, trim most punctuations
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = ''.join(c for c in s if c not in punctuation)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

#Load data, process data into pairs(pair[0] is current sentence, pair[1] is the next sentence)
#Pair[0] is the input sentence, pair[1] is the ground truth
def loadData():
    pf = open('pop_eng_clean.csv', 'r')
    df = pd.read_csv(pf, delimiter=',', names=['index', 'remix', 'yead', 'singer', 'type', 'lyric','keywords','scores'])
    lyrics = df['lyric'].tolist()
    keywords = df['keywords'].tolist()
    scores = df['scores'].tolist()
    print(len(keywords))
    
    pairs = []
    cnt = 0
    songs_len = len(lyrics)
    total_line = 0
    for i in range(1, len(lyrics)):
        if(i%1000==0): print(i)
#     for i in range(1, 1000):
        lyrics_lines = lyrics[i].split('\n')
        keywords_lines = keywords[i].split('\n')
        scores_lines = scores[i].split('\n')
        lyric_line_num = len(lyrics_lines)
#         keywords_line_num = len(keywords_lines)
#         in each song
        isFirstLine = True
#     sometimes in the dataset there are 49 lines of lyrics, only 1 keywords
        if(len(keywords_lines)<len(lyrics_lines)): continue
#         print(len(lyrics_lines), len(keywords_lines))
        for j in range(lyric_line_num):
            total_line+=1
            pair = []
            current_lyric = lyrics_lines[j]
            current_keyword = keywords_lines[j]
            current_score = scores_lines[j]
            if(len(current_score)>0 and round(float(current_score))>20):
                if(isFirstLine):
                    pair.append(normalizeString(current_keyword))
                    isFirstLine = False
                else:
                    pair.append(normalizeString(current_keyword+' '+previous_lyric))
                pair.append(normalizeString(current_lyric))
                pairs.append(pair)
            previous_lyric = current_lyric
    print(total_line)
    print(len(pairs))
    return pairs
            
            

    
#     for d in lyric:
#         if(cnt==0):break
#         cnt+=1
#         lines = d.decode('utf-8').split('\n')
#         #Process the first pair [keyword + null]
#         first_keyword = SigKeywordExtract(lines[0])
#         if first_keyword:
#             pair = [first_keyword[0][0].encode('utf-8'), lines[0].encode('utf-8')]
#         i = 0
#         j = 1
#         pairs.append(pair)

#         while j<len(lines):
#             pair = []
#             next_keyword = SigKeywordExtract(lines[j])
#             if next_keyword:
#                 if(next_keyword[0][1]>40):
#                     newline = next_keyword[0][0] + ' ' + normalizeString(lines[i])
#                     pair.append(normalizeString(newline).encode('utf-8'))
#                     pair.append(normalizeString(lines[j]).encode('utf-8'))
#                     pairs.append(pair)
#             i += 1
#             j += 1
#         break
#     return pairs

In [15]:
#Define a prepareData function to process all the input sentence pairs
def prepareData(lang):
    pairs = loadData()
    lang = Lang(lang)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        if pair: 
            lang.addSentence(pair[0])
            lang.addSentence(pair[1])
    print("Counted words:")
    print(lang.name, lang.n_words)
    return lang, pairs


lang, pairs = prepareData('lyric')
#randomly choose a pair, print pair[0] and pair[1]
i = 0
# while i<100:
#     print(random.choice(pairs))
#     i += 1

28806
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
1156621
797532
Read 797532 sentence pairs
Counting words...
Counted words:
lyric 96593


In [16]:
selected_pairs = pairs

In [17]:
#Define Encoder and Decoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [42]:
#Define Encoder and Decoder
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.lstm(output.view(len(input), 1, -1), hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
    
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.lstm(output.view(len(input), 1, -1), hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [43]:
import torch.utils.data
#Prepare for training data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0])
    target_tensor = tensorFromSentence(lang, pair[1])
    return (input_tensor, target_tensor)

In [44]:
teacher_forcing_ratio = 1

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [45]:
# Define a function to plot the figure
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

#define epoch for training and print the result periodically 
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(selected_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [46]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [67]:
hidden_size = 256
cell_size = 256
encoder1 = EncoderRNN(lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, lang.n_words).to(device)

encoder2 = EncoderLSTM(lang.n_words, hidden_size).to(device)
decoder2 = DecoderLSTM(hidden_size, lang.n_words).to(device)

trainIters(encoder2, decoder2, 100000, print_every=1000, learning_rate=0.1)
torch.save(encoder2.state_dict(), "encoder2")

1m 1s (- 100m 51s) (1000 1%) 7.5366
1m 48s (- 88m 44s) (2000 2%) 6.7303
2m 35s (- 83m 49s) (3000 3%) 6.4894
3m 24s (- 81m 41s) (4000 4%) 6.2938
4m 13s (- 80m 24s) (5000 5%) 6.1339
5m 2s (- 78m 53s) (6000 6%) 6.2620
5m 50s (- 77m 43s) (7000 7%) 5.9556
6m 40s (- 76m 46s) (8000 8%) 6.0603
7m 28s (- 75m 34s) (9000 9%) 6.0251
8m 17s (- 74m 37s) (10000 10%) 5.9799
9m 6s (- 73m 39s) (11000 11%) 5.9551
9m 55s (- 72m 45s) (12000 12%) 5.8077
10m 44s (- 71m 52s) (13000 13%) 5.8317
11m 31s (- 70m 49s) (14000 14%) 5.8142
12m 19s (- 69m 52s) (15000 15%) 5.7716
13m 8s (- 69m 1s) (16000 16%) 5.8118
13m 57s (- 68m 10s) (17000 17%) 5.7200
14m 45s (- 67m 13s) (18000 18%) 5.7098
15m 33s (- 66m 20s) (19000 19%) 5.7472
16m 21s (- 65m 25s) (20000 20%) 5.7608
17m 11s (- 64m 42s) (21000 21%) 5.7077
18m 29s (- 65m 33s) (22000 22%) 5.6598
20m 0s (- 67m 0s) (23000 23%) 5.6188
21m 34s (- 68m 17s) (24000 24%) 5.6622


KeyboardInterrupt: 

In [ ]:
torch.save(decoder2.state_dict(), "decoder2")

In [ ]:
encoder2 = EncoderLSTM(lang.n_words, hidden_size, cell_size).to(device)
decoder2 = DecoderLSTM(hidden_size, cell_size, lang.n_words).to(device)

encoder2.load_state_dict(torch.load("./encoder2"))
decoder2.load_state_dict(torch.load("./decoder2"))

trainIters(encoder2, decoder2,5000, print_every=1000, learning_rate=0.1)

In [ ]:
trainIters(encoder2, decoder2,20000, print_every=1000, learning_rate=0.05)

In [ ]:
trainIters(encoder2, decoder2,10000, print_every=1000, learning_rate=0.1)

In [ ]:
encoder2 = EncoderLSTM(lang.n_words, hidden_size, cell_size).to(device)
decoder2 = DecoderLSTM(hidden_size, cell_size, lang.n_words).to(device)

In [ ]:
encoder2.load_state_dict(torch.load('encoder2'))
decoder2.load_state_dict(torch.load('decoder2'))

In [48]:
#temporary function
def evaluate(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
       # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            #decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words


In [63]:
# input_sen = "i am gonna be happy one day <EOS>"
input_sen = "i am gonna be happy one day"
output_num = 3
keywords = list(rake.sen_extract(input_sen, word_num=output_num))
print ("Initial keywords: " + str(keywords))

for keyword in keywords:
    try:
        lang.word2index[keyword]
    except KeyError:
        keywords.remove(keyword)
        
print ("Filtered keywords: " + str(keywords))
output_num = len(keywords)

Initial keywords: ['happi', 'day', 'gon']
Filtered keywords: ['happi', 'day', 'gon']


In [64]:
def indexesFromWord(lang, word):
    return [lang.word2index[word.encode('utf-8')]]

def tensorFromWord(lang, sentence):
    indexes = indexesFromWord(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [65]:
def evaluate_word(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = tensorFromWord(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
       # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            #decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [66]:
output_words = []
input_sen = normalizeString(input_sen)
new_input = keywords[0] + " " + input_sen
output_word = evaluate(encoder1, decoder1, new_input)
print ("First sentence: " + str(output_word))
output_words.append(output_word)
for i in range(1, output_num):
    new_sen = keywords[i] + ' ' + ' '.join(output_word)
    sen = ' '.join(new_sen.split(' ')[:-1])
    output_word = evaluate(encoder1, decoder1, sen)
    output_words.append(output_word)

    
print ("Final result: " + str(output_words))

First sentence: ['cemetri', 'lohk', 'concentrating', 'crafti', 'bongo', 'breads', 'eastsid', 'Aimin', 'Jealousies', 'keojyeo']
Final result: [['cemetri', 'lohk', 'concentrating', 'crafti', 'bongo', 'breads', 'eastsid', 'Aimin', 'Jealousies', 'keojyeo'], ['Tabitha', 'musiques', 'inneun', 'inneun', 'mournin', 'mennar', 'poin', 'geotman', 'Butmost', 'change'], ['lohk', 'concentrating', 'crafti', 'bongo', 'glint', 'mennar', 'Muffys', 'mennar', 'silvergirl', 'Etta']]
